In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# read in csv file for user data nand movie data
data = pd.io.parsers.read_csv('/content/drive/Shareddrives/Math 365 CLA final project/data/ratings.dat', 
    names=['user_id', 'movie_id', 'rating', 'time'],
    engine='python', delimiter='::')
movie_data = pd.io.parsers.read_csv('/content/drive/Shareddrives/Math 365 CLA final project/data/movies.dat',
    names=['movie_id', 'title', 'genre'],
    engine='python', delimiter='::',encoding='latin-1')



In [ ]:
# from wenxuan/ do not run this chunk when testing
data = pd.io.parsers.read_csv('https://drive.google.com/file/d/1EoSOzI808qcjIUBUowYIElE3OsCms9Nu/view?usp=share_link', 
    names=['user_id', 'movie_id', 'rating', 'time'],
    engine='python', delimiter='::')

movie_data = pd.io.parsers.read_csv('https://drive.google.com/file/d/1W1r5s3E8y2SqSf8ZA9kndp-zw8r5EbzX/view?usp=share_link',
    names=['movie_id', 'title', 'genre'],
    engine='python', delimiter='::',encoding='latin-1')

In [ ]:
# create the rating matrix and normalize the matrix
ratings_mat = np.ndarray(
    shape=(np.max(data.movie_id.values), np.max(data.user_id.values)),
    dtype=np.uint8)
ratings_mat[data.movie_id.values-1, data.user_id.values-1] = data.rating.values

normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T

In [ ]:
#Computing the Singular Value Decomposition (SVD)
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
# Compute the singular values (SVD)
U, S, V = np.linalg.svd(A)

#Function to calculate the cosine similarity (sorting by most similar and returning the top N)
def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id - 1 # Movie id starts from 1 in the dataset
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

# Function to print top N similar movies
def print_similar_movies(movie_data, movie_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movie_data[movie_data.movie_id == movie_id].title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.movie_id == id].title.values[0])

#k-principal components to represent movies, movie_id to find recommendations, top_n print n results        
k = 50
movie_id = 10 # (getting an id from movies.dat)
top_n = 10
sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, movie_id, top_n)

#Printing the top N similar movies
print_similar_movies(movie_data, movie_id, indexes)

Recommendations for GoldenEye (1995): 

GoldenEye (1995)
Paralyzing Fear: The Story of Polio in America, A (1998)
Windows (1980)
Naked Man, The (1998)
Second Jungle Book: Mowgli & Baloo, The (1997)
Santa with Muscles (1996)
Bye-Bye (1995)
Soft Toilet Seats (1999)
Legal Deceit (1997)
Destiny Turns on the Radio (1995)


<ipython-input-10-cf2c39161de3>:10: RuntimeWarning: invalid value encountered in true_divide
  similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)


In [ ]:
# note
"""
前提条件是matrix是complete的，然后根据SVD里面的V去做cosine similarity去做recommendation
"""